# Baseline Model

## Table of Contents
1. [Model Choice](#model-choice)
2. [Feature Selection](#feature-selection)
3. [Implementation](#implementation)
4. [Evaluation](#evaluation)


## Model Choice

[Explain why you've chosen a particular model as the baseline. This could be a simple statistical model or a basic machine learning model. Justify your choice.]


## Feature Selection

[Indicate which features from the dataset you will be using for the baseline model, and justify your selection.]


## Implementation

[Implement your baseline model here.]



In [ ]:
# Initialize the baseline model
# Your implementation code here
# Define the model
import tensorflow as tf
#from tensorflow.keras import layers
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

# Print the model summary
model.summary()


In [ ]:
# Define the paths to the folders containing the image data
train_data_dir = '../Dataset/cleaned_scaled_split/train/'
validation_data_dir = '../Dataset/cleaned_scaled_split/val/'
test_data_dir = '../Dataset/cleaned_scaled_split/test/'

# Set the parameters for image data generation
batch_size = 64
img_height = 256
img_width = 256
# Load the training data from the folders
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory=train_data_dir,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=64,
    image_size=(256, 256))


# Load the validation data from the folders
validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory=validation_data_dir,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=64,
    image_size=(256, 256))

# Load the validation data from the folders
test_ds = tf.keras.utils.image_dataset_from_directory(
    directory=test_data_dir,
    labels='inferred',
    label_mode='int',
    color_mode='rgb',
    batch_size=64,
    image_size=(256, 256))

In [ ]:

# # Get an element from the train dataset
# element = next(iter(train_ds.take(1)))

# # Access the input and target tensors
# input_tensor, target_tensor = element

# # Print the shapes of the tensors
# print("Input tensor shape:", input_tensor.shape)
# print("Target tensor shape:", target_tensor.shape)



In [ ]:
# # Train the network using the loaded data
import time
name = "training_" + time.strftime("%Y%m%d-%H%M")
csv_logger =  tf.keras.callbacks.CSVLogger(name+'.csv', append=True, separator=';')
model.fit(train_ds, epochs=10, validation_data=validation_ds, callbacks=[csv_logger])
print("Training complete!")

In [ ]:
# Save the final model parameters to a file
#model.save_weights('baseline_model_weights_01.ckpt')
model.save('baseline_model_10_epoch_tvt_split.keras')

In [ ]:
import os

train_dir = '../Dataset/train_cleaned_scaled/train/'

datadirs = os.listdir(train_dir)
print(datadirs)

In [ ]:
import matplotlib.pyplot as plt

train_acc = model.history.history['accuracy']
plt.figure(figsize=(8, 4), dpi=150)
plt.plot(train_acc)
plt.grid()
plt.xticks(range(0, 10))
plt.xlabel('Epoch')
plt.ylabel('Accuracy')

## Evaluation

[Clearly state what metrics you will use to evaluate the model's performance. These metrics will serve as a starting point for evaluating more complex models later on.]



In [ ]:
import numpy as np
model = tf.keras.models.load_model('baseline_model_10_epoch_tvt_split.keras')	

class_correct = [0]*10
class_counters = [0]*10
for batch in test_ds:
    images, labels = batch
    predictions = model.predict(images)
    y_pred_classes = np.argmax(predictions, axis=1)
    print(np.mean(y_pred_classes==labels.numpy()))
    for i in range(10):
        class_correct[i] += np.sum(y_pred_classes[labels.numpy()==i] == i)
        class_counters[i] += np.sum(labels.numpy()==i)

calss_accuracy = [class_correct[i]/class_counters[i] for i in range(10)]



# # For a regression problem, you might use:
# # mse = mean_squared_error(y_test, y_pred)

# # Your evaluation code here


In [ ]:
import matplotlib.pyplot as plt

# Define the class labels
class_labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Plot the bar chart
plt.figure(figsize=(8, 4), dpi=150)
plt.grid()
plt.bar(class_labels, np.asarray(calss_accuracy)*100)

# Add labels and title
#plt.xlabel('Class')
plt.ylabel('Accuracy in %')
plt.title('Accuracy of the baseline model on the test data for each class')

plt.ylim(0, 100)
plt.xticks(class_labels,datadirs, rotation=45, ha='right')
# Show the plot
plt.show()